In [1]:
import pandas as pd
import jsonlines
import os
from glob import glob

In [40]:
files = glob("../outputs/*/*/prometheus-13b-v1.0/*.json")
dfs = []

def filter_score(x):
    if x["score"] > 0:
        return x["score"]
    if "overall score is" in x["critic"]:
        score = int(x["critic"].split("overall score is")[1].split(".")[0].strip().replace(",","").split(" ", 1)[0])
        return score
    elif "a score of " in x["critic"]:
        score = x["critic"].split("a score of ")[1].split(" ")[0].split(".")[0].split(",")[0].strip()
        if score:
            return int(score)
        else:
            return -1
    else:
        return -1

for file in files:
    df = pd.read_json(file, lines=True)
    df["score"] = df.judge.apply(filter_score)
    dfs.append(df)

df = pd.concat(dfs)
df.shape

(10009, 8)

In [42]:
df["output_len"] = df.output.apply(len)
df.head()
# df[df.score < 0].judge.apply(lambda x: x['critic']).tolist()


,instruction,output,generator,dataset,judge,judge_model,score,output_len
0,What are the names of some famous actors that ...,Some famous actors that started their careers ...,heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epo...,helpful_base,"{'critic': ' Based on the scoring rubric, the ...",kaist-ai/prometheus-13b-v1.0,1,542
1,How did US states get their names?,The names of many US states are derived from E...,heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epo...,helpful_base,"{'critic': ' Based on the scoring rubric, the ...",kaist-ai/prometheus-13b-v1.0,1,933
2,"Hi, my sister and her girlfriends want me to p...",Sure! Here's a step-by-step breakdown of kickb...,heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epo...,helpful_base,"{'critic': ' Based on the scoring rubric, the ...",kaist-ai/prometheus-13b-v1.0,1,1780
3,What is some cool music from the 1920s?,Cool music from the 1920s may include:\n\n1. T...,heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epo...,helpful_base,"{'critic': ' Based on the scoring rubric, the ...",kaist-ai/prometheus-13b-v1.0,1,969
4,How do I wrap a present neatly?,"Yes, here are some tips to help you wrap a pre...",heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epo...,helpful_base,"{'critic': ' Based on the scoring rubric, the ...",kaist-ai/prometheus-13b-v1.0,2,998


In [36]:
df[df.score > 0].groupby("generator").agg({
    "score": ["mean", "median", "std", "count"],
})

score                   \
                                                        mean median       std   
generator                                                                       
heegyu/TinyLlama-1.1b-feedback-all@epoch-1          1.785803    1.0  1.312304   
heegyu/TinyLlama-1.1b-feedback-all@epoch-2          1.520548    1.0  1.068951   
heegyu/TinyLlama-1.1b-feedback-all@epoch-3          1.432130    1.0  1.013039   
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-1  1.829390    1.0  1.296276   
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-2  1.737888    1.0  1.256671   
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-3  1.603727    1.0  1.177414   
heegyu/TinyLlama-1.1b-feedback-tree-3-epoch3-di...  1.576000    1.0  1.131967   
heegyu/TinyLlama-1.1b-max-margin@epoch-2            1.618634    1.0  1.170704   
heegyu/TinyLlama-1.1b-max-margin@epoch-3            1.513682    1.0  1.085882   
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...  1.869240    1.0  1.373634   
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...  1.898010    1.0  1.386484   
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...  1.777086    1.0  1.289270   
text_davinci_003                                    1.778882    1.0  1.263062   

                                                          
                                                   count  
generator                                                 
heegyu/TinyLlama-1.1b-feedback-all@epoch-1           803  
heegyu/TinyLlama-1.1b-feedback-all@epoch-2           803  
heegyu/TinyLlama-1.1b-feedback-all@epoch-3           803  
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-1   803  
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-2   805  
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-3   805  
heegyu/TinyLlama-1.1b-feedback-tree-3-epoch3-di...   250  
heegyu/TinyLlama-1.1b-max-margin@epoch-2             805  
heegyu/TinyLlama-1.1b-max-margin@epoch-3             804  
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...   803  
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...   804  
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...   803  
text_davinci_003                                     805

In [44]:
df[df.score > 0].groupby("generator").agg({
    "score": "mean",
    "output_len": ["mean", "median", "std", "count"],
})

score   output_len  \
                                                        mean         mean   
generator                                                                   
heegyu/TinyLlama-1.1b-feedback-all@epoch-1          1.784826  1156.945274   
heegyu/TinyLlama-1.1b-feedback-all@epoch-2          1.520548  1336.073474   
heegyu/TinyLlama-1.1b-feedback-all@epoch-3          1.436567  1274.685323   
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-1  1.829390  1035.331258   
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-2  1.737888   981.081988   
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-3  1.603727  1107.804969   
heegyu/TinyLlama-1.1b-feedback-tree-3-epoch3-di...  1.558739  1086.747851   
heegyu/TinyLlama-1.1b-max-margin@epoch-2            1.618634  1237.853416   
heegyu/TinyLlama-1.1b-max-margin@epoch-3            1.513682  1294.924129   
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...  1.869240  1041.265255   
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...  1.901863  1040.172671   
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...  1.776119  1133.180348   
text_davinci_003                                    1.778882  1059.672050   

                                                                              
                                                    median         std count  
generator                                                                     
heegyu/TinyLlama-1.1b-feedback-all@epoch-1           990.5  901.824627   804  
heegyu/TinyLlama-1.1b-feedback-all@epoch-2          1195.0  918.657444   803  
heegyu/TinyLlama-1.1b-feedback-all@epoch-3          1136.0  909.712958   804  
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-1   893.0  800.633616   803  
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-2   797.0  793.368868   805  
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-3   918.0  879.540126   805  
heegyu/TinyLlama-1.1b-feedback-tree-3-epoch3-di...   920.0  853.469184   349  
heegyu/TinyLlama-1.1b-max-margin@epoch-2            1107.0  866.385656   805  
heegyu/TinyLlama-1.1b-max-margin@epoch-3            1169.5  889.523366   804  
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...   892.0  825.778962   803  
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...   897.0  822.370682   805  
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...   987.0  878.581902   804  
text_davinci_003                                     956.0  783.421212   805